![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)

## Options Data Analysis Strategy
This notebook demonstrates how to analyze options data one timeframe at a time.

In [ ]:
# Import required libraries
from QuantConnect import *
from QuantConnect.Data import *
from QuantConnect.Data.Market import *
from IPython.display import clear_output
import time

class OptionsAnalysis(QCAlgorithm):
    
    def Initialize(self):
        self.SetStartDate(2024, 1, 1)
        self.SetEndDate(2024, 1, 2)  # Just analyzing one day for now
        
        # Set SPY as the underlying with hourly resolution
        equity = self.AddEquity("SPY", Resolution.Hour)
        # Add options with hourly resolution
        option = self.AddOption("SPY", Resolution.Hour)
        
        # Use the default filter for now
        option.SetFilter(-2, +2, timedelta(0), timedelta(30))
        
        self.last_time = None
    
    def OnData(self, slice):
        if not slice.OptionChains: return
        
        # Get the options chain
        for chain in slice.OptionChains.Values:
            current_time = chain.Time
            
            # Only process if this is a new timeframe
            if self.last_time is not None and current_time == self.last_time:
                continue
                
            # Clear previous output
            clear_output(wait=True)
            
            print(f"\nTimeframe: {current_time}")
            print("-" * 80)
            print("Strike   Call Bid   Call Ask   Put Bid    Put Ask")
            print("-" * 80)
            
            # Sort contracts by strike price
            contracts = sorted(chain, key=lambda x: x.Strike)
            
            # Display data for each strike price
            for contract in contracts:
                if contract.Right == OptionRight.Call:
                    call = contract
                    # Find the corresponding put
                    put = next((x for x in chain if x.Strike == call.Strike and x.Right == OptionRight.Put), None)
                    if put:
                        print(f"{call.Strike:8.2f} {call.BidPrice:9.2f} {call.AskPrice:9.2f} "
                              f"{put.BidPrice:9.2f} {put.AskPrice:9.2f}")
            
            self.last_time = current_time
            # Add a small delay to make the output readable
            time.sleep(1)

# Create and run the algorithm
qc = QuantConnect.Python.Initialize()
algorithm = OptionsAnalysis()
qc.SetAlgorithm(algorithm)
results = qc.Run()